# Goals for today
- Defining new types in Julia
- Abstract types
- Generic code

# Defining new types in Julia

In [1]:
mutable struct MyDiscreteWalker # -- camel case
    x::Int64
end

Julia automatically created 2 **constructors** &rarr; functions to create objects of that types

In [2]:
methods(MyDiscreteWalker)

# 2 methods for type constructor:
[1] MyDiscreteWalker(x::Int64) in Main at In[1]:2
[2] MyDiscreteWalker(x) in Main at In[1]:2

In [3]:
w = MyDiscreteWalker(3)

MyDiscreteWalker(3)

In [4]:
typeof(w)

MyDiscreteWalker

In [5]:
Complex(3, 5)

3 + 5im

In [7]:
@which Complex(3,5)

Complex(re::T, im::T) where T<:Real in Base at complex.jl:12

In [8]:
w isa MyDiscreteWalker # w is an instance of the type MyDiscreteWalker

true

`MyDiscreteWalker(x::Int64)` is a method that works *only* when it receives an argument `x` of type `Int64`

`MyDiscreteWalker(x)` will accept an `x` of any type

In [10]:
MyDiscreteWalker(4.0)

MyDiscreteWalker(4)

In [11]:
MyDiscreteWalker(4.1)

LoadError: InexactError: Int64(4.1)